In [27]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup

## I. Thu thập dữ liệu:

In [28]:
link = 'https://www.worldometers.info/coronavirus/'
page = requests.get(link)
page.status_code

200

In [29]:
soup = BeautifulSoup(page.content)

In [30]:
table = soup.find('table', attrs={'id': 'main_table_countries_yesterday'})

In [31]:
rows = table.find_all("tr", attrs={"style": ""})

In [32]:
data = []
for i,item in enumerate(rows):
    if i == 0:
        data.append(item.text.strip().split("\n")[1:17])
        data[0][12]+=data[0][13]
        data[0]=data[0][:13]+data[0][-1:]
        data[0].append("Area")
    else:
        data.append(item.text.strip().split("\n")[1:16])

In [33]:
df = pd.DataFrame(data)
df = pd.DataFrame(data[2:-1], columns=data[0][:15])
df = df.rename(columns={'Tot\xa0Cases/1M pop':'TotCases/1M pop'})
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Area
0,China,"115,466","+1,938","4,636",,"103,600",+169,"7,230",6,80,3,"160,000,000","111,163","1,439,323,776",Asia
1,USA,"81,174,677","+5,143","993,811",+118,"56,071,103","+155,965","24,109,763","4,250","242,828","2,973","964,386,655","2,884,885","334,289,437",North America
2,India,"42,993,494","+2,503","515,907",+57,"42,441,449","+4,377","36,138","8,944","30,644",368,"778,520,151","554,906","1,402,976,534",Asia
3,Brazil,"29,368,776","+18,397","655,139",+146,"27,742,324","+70,731","971,313","8,318","136,524","3,045","63,776,166","296,471","215,117,939",South America
4,France,"23,514,144","+60,422","140,109",+29,"22,163,944","+17,144","1,210,091","1,928","358,895","2,138","246,629,975","3,764,303","65,518,106",Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Macao,82,,,,79,,3,,123,,"5,318","8,005","664,320",Asia
218,Samoa,48,+2,,,28,,20,,239,,,,"200,646",Australia/Oceania
219,Vatican City,29,,,,28,,1,,"36,070",,,,804,Europe
220,Western Sahara,10,,1,,8,,1,,16,2,,,"622,186",Africa


## II. Tiền xử lý dữ liệu:

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Country,Other     222 non-null    object
 1   TotalCases        222 non-null    object
 2   NewCases          222 non-null    object
 3   TotalDeaths       222 non-null    object
 4   NewDeaths         222 non-null    object
 5   TotalRecovered    222 non-null    object
 6   NewRecovered      222 non-null    object
 7   ActiveCases       222 non-null    object
 8   Serious,Critical  222 non-null    object
 9   TotCases/1M pop   222 non-null    object
 10  Deaths/1M pop     222 non-null    object
 11  TotalTests        222 non-null    object
 12  Tests/1M pop      222 non-null    object
 13  Population        222 non-null    object
 14  Area              222 non-null    object
dtypes: object(15)
memory usage: 26.1+ KB


In [35]:
def replaceString(s:str):
    s = s.replace('+', '')
    s = s.replace(',', '')
    s = s.strip()
    return s

for col in df.columns:
    df[col] = df[col].apply(lambda s: replaceString(s))

In [36]:
df = df.replace(['', 'N/A'], np.nan)
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",TotCases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Area
0,China,115466,1938,4636,NaN,103600,169,7230,6,80,3,160000000,111163,1439323776,Asia
1,USA,81174677,5143,993811,118,56071103,155965,24109763,4250,242828,2973,964386655,2884885,334289437,North America
2,India,42993494,2503,515907,57,42441449,4377,36138,8944,30644,368,778520151,554906,1402976534,Asia
3,Brazil,29368776,18397,655139,146,27742324,70731,971313,8318,136524,3045,63776166,296471,215117939,South America
4,France,23514144,60422,140109,29,22163944,17144,1210091,1928,358895,2138,246629975,3764303,65518106,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Macao,82,NaN,NaN,NaN,79,NaN,3,NaN,123,NaN,5318,8005,664320,Asia
218,Samoa,48,2,NaN,NaN,28,NaN,20,NaN,239,NaN,NaN,NaN,200646,Australia/Oceania
219,Vatican City,29,NaN,NaN,NaN,28,NaN,1,NaN,36070,NaN,NaN,NaN,804,Europe
220,Western Sahara,10,NaN,1,NaN,8,NaN,1,NaN,16,2,NaN,NaN,622186,Africa


In [37]:
columns = df.columns
for i in range(1, len(columns) - 1):
    df[columns[i]] = df[columns[i]].astype(float)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country,Other     222 non-null    object 
 1   TotalCases        222 non-null    float64
 2   NewCases          132 non-null    float64
 3   TotalDeaths       216 non-null    float64
 4   NewDeaths         83 non-null     float64
 5   TotalRecovered    210 non-null    float64
 6   NewRecovered      112 non-null    float64
 7   ActiveCases       211 non-null    float64
 8   Serious,Critical  161 non-null    float64
 9   TotCases/1M pop   222 non-null    float64
 10  Deaths/1M pop     216 non-null    float64
 11  TotalTests        211 non-null    float64
 12  Tests/1M pop      211 non-null    float64
 13  Population        222 non-null    float64
 14  Area              222 non-null    object 
dtypes: float64(13), object(2)
memory usage: 26.1+ KB


In [39]:
df.to_csv(r'./data.csv',index=False)